In [2]:
from __future__ import division
import cv2
import numpy as np
import matplotlib.pyplot as plt
import csv

try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract
import os
import random

In [3]:
KnownErrors = {'+':'-',
               '~':'-',
               'S':'5',
               'i':'1',
               '&':'8',
               'B':'8',
               '?':'2',
               '$':'8',
               'g':'8',
               'A':'4',
               'l':'1',
               '.':''}

Numbers = ['0','1','2','3','4','5','6','7','8','9','-']

weirdCharacters = {}

def convert(s): 
  
    # initialization of string to "" 
    str1 = "" 
  
    # using join function join the list s by  
    # separating words by str1 
    return(str1.join(s)) 

def cleanRow(row,filename):
    Extracted = True
    notReadable = []
    for i in range(len(row)):
        try: 
            row[i] = int(row[i])
        except:
            notReadable.append(i)
    if len(notReadable) == 0:
        Extracted = True
    if len(notReadable) == 1:
        Extracted = True
        MissScanIndex = notReadable[0]
        if MissScanIndex == len(row) -1:
            row[MissScanIndex] = sum(row[:-1])
        else:
            row[MissScanIndex] = row[-1] - sum(row[:MissScanIndex]) - sum(row[MissScanIndex+1:-1])
    elif len(notReadable) > 1:
        Extracted = False
        for i in notReadable:
            rowList = list(row[i])
            for j in range(len(rowList)):
                if not (rowList[j] in Numbers):
                    if rowList[j] in KnownErrors.keys():
                        rowList[j] = KnownErrors[rowList[j]]
                    else:
                        # Update statistics of weird characters
                        if rowList[j] in weirdCharacters.keys():
                            weirdCharacters[rowList[j]] += 1
                        else:
                            weirdCharacters[rowList[j]] = 1
                        #if rowList[j] == 'a':
                         #   print row,filename
                        rowList[j] = ''
            combNumb = convert(rowList)
            try:
                row[i] = int(combNumb)
            except:
                pass
        notReadable = []
        for i in range(len(row)):
            try: 
                row[i] = int(row[i])
            except:
                notReadable.append(i)
        if len(notReadable) == 0:
            Extracted = True
        if len(notReadable) == 1:
            Extracted = True
            MissScanIndex = notReadable[0]
            #print row, MissScanIndex,row[-1],sum(row[:MissScanIndex]
            if MissScanIndex == len(row) -1:
                row[MissScanIndex] = sum(row[:-1])
            else:
                row[MissScanIndex] = row[-1] - sum(row[:MissScanIndex]) - sum(row[MissScanIndex+1:-1])
    # Check the sum
    if Extracted and row != []:
        if np.sum(row) != 2 * row[-1]:
            if np.sum(row) == - 2 * row[-1]:
                row[-1] = -row[-1]
            else:
                diff = np.sum(row) - 2 * row[-1]
                if diff % 2 == 0:
                    diff = diff / 2
                    for i in range(len(row[:-1])):
                        if row[i] == diff:
                            row[i] = - row[i]
                            break
    if Extracted and row != []:
        if np.sum(row) != 2 * row[-1]:
            row = None
    else:
        row = None
    return row

def shuffle(a,pos1,pos2):
    a[pos1], a[pos2] = a[pos2], a[pos1] 
    return a

def cotemplateTable(table):
    newTable = table
    LT = len(table)
    LR = len(table[0])-1
    NR = LR // 2
    index = range(LT)
    randomID = random.shuffle(index)
    for i in range(10-LT):
        pos = i % LT
        RandRow = np.copy(table[index[i % LT]])
        if LR>2:
            for j in range(NR):
                pos = random.randint(0,LR-3)
                RandRow = shuffle(RandRow,pos,pos+1)
        else:
            for j in range(NR):
                pos = random.randint(0,LR-2)
                RandRow[pos] += random.randint(1,3) - 2
            RandRow[-1] = sum(RandRow[:-1])
        newTable.append(list(RandRow))
    return newTable


Correct = 0
Wrong = 0
files = {}
for filename in os.listdir(os.getcwd()):
    if filename[-3:] == 'txt':
        with open(filename) as csvfile:
            lines = csv.reader(csvfile,delimiter=',')
            table = []
            for row in lines:
                ProcessedRow = cleanRow(row,filename)
                if ProcessedRow != None:
                    table.append(ProcessedRow)
                    Correct += 1
                else:
                    Wrong += 1
        #### CHECK THE TABLE LENGTH #####
        files[filename] = 10 - len(table)
        if len(table) < 10:
            table = cotemplateTable(table)
        if table != []:
            newName = 'Corrected/'+filename
            with open(newName, 'w') as file:
                writer = csv.writer(file)
                writer.writerows(table)
                    
print Correct, 'Correct', int(round(Correct / 92.3)),'%' 
a = weirdCharacters.items()
sortFiles = sorted(files.items(), key= lambda x:x[1], reverse=True)
for i in sortFiles:
    print i


8686 Correct 94 %
('87.3.txt', 8)
('95.3.txt', 7)
('65.3.txt', 7)
('115.4.txt', 5)
('109.3.txt', 5)
('91.4.txt', 5)
('147.6.txt', 5)
('89.3.txt', 5)
('78.3.txt', 4)
('115.3.txt', 4)
('95.4.txt', 4)
('91.3.txt', 4)
('64.4.txt', 4)
('124.3.txt', 4)
('109.4.txt', 4)
('137.3.txt', 4)
('87.2.txt', 4)
('105.5.txt', 4)
('113.3.txt', 3)
('140.3.txt', 3)
('97.2.txt', 3)
('105.2.txt', 3)
('134.3.txt', 3)
('109.2.txt', 3)
('142.4.txt', 3)
('113.2.txt', 3)
('115.2.txt', 3)
('151.4.txt', 3)
('89.4.txt', 3)
('77.5.txt', 2)
('131.1.txt', 2)
('46.4.txt', 2)
('113.4.txt', 2)
('93.2.txt', 2)
('167.2.txt', 2)
('44.3.txt', 2)
('66.4.txt', 2)
('122.1.txt', 2)
('107.3.txt', 2)
('46.1.txt', 2)
('13.5.txt', 2)
('52.3.txt', 2)
('84.5.txt', 2)
('94.3.txt', 2)
('28.4.txt', 2)
('124.1.txt', 2)
('93.3.txt', 2)
('129.3.txt', 2)
('60.3.txt', 2)
('108.3.txt', 2)
('85.3.txt', 2)
('44.2.txt', 2)
('97.4.txt', 2)
('155.1.txt', 2)
('174.1.txt', 2)
('105.3.txt', 2)
('22.2.txt', 2)
('175.1.txt', 2)
('68.2.txt', 2)
('88.3.tx

In [4]:
5 % 2


1

In [ ]:
a = [1,2,3,4,5,6,7,8,9]
b = a[:5] + a[6] + a[5] + a[]